In [1]:
using CSV
using DataFrames
using Statistics

In [2]:
df = CSV.read("Quality of Service 5G.csv", DataFrame)

Row,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation
,String15,String15,String31,String15,String7,String15,String15,String3
1,9/3/2023 10:00,User_1,Video_Call,-75 dBm,30 ms,10 Mbps,15 Mbps,70%
2,9/3/2023 10:00,User_2,Voice_Call,-80 dBm,20 ms,100 Kbps,120 Kbps,80%
3,9/3/2023 10:00,User_3,Streaming,-85 dBm,40 ms,5 Mbps,6 Mbps,75%
4,9/3/2023 10:00,User_4,Emergency_Service,-70 dBm,10 ms,1 Mbps,1.5 Mbps,90%
5,9/3/2023 10:00,User_5,Online_Gaming,-78 dBm,25 ms,2 Mbps,3 Mbps,85%
6,9/3/2023 10:00,User_6,Background_Download,-90 dBm,50 ms,500 Kbps,550 Kbps,70%
7,9/3/2023 10:00,User_7,Web_Browsing,-88 dBm,30 ms,1 Mbps,1 Mbps,60%
8,9/3/2023 10:00,User_8,IoT_Temperature,-95 dBm,100 ms,10 Kbps,15 Kbps,50%
9,9/3/2023 10:00,User_9,Video_Streaming,-82 dBm,35 ms,3 Mbps,3.5 Mbps,80%


In [3]:
println("\nMissing values per column:")
println(sum.(ismissing.(eachcol(df))))


Missing values per column:
[0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Nothing,InlineSt…,Nothing,InlineSt…,Int64,DataType
1,Timestamp,,9/3/2023 10:00,,9/3/2023 10:06,0,String15
2,User_ID,,User_1,,User_99,0,String15
3,Application_Type,,Background_Download,,Web_Browsing,0,String31
4,Signal_Strength,,-100 dBm,,-99 dBm,0,String15
5,Latency,,0 ms,,99 ms,0,String7
6,Required_Bandwidth,,0 Kbps,,95 Kbps,0,String15
7,Allocated_Bandwidth,,0 Kbps,,95 Kbps,0,String15
8,Resource_Allocation,,50%,,90%,0,String3


In [5]:
df.Resource_Allocation = parse.(Float64, replace.(df.Resource_Allocation, "%" => "")) / 100

400-element Vector{Float64}:
 0.7
 0.8
 0.75
 0.9
 0.85
 0.7
 0.6
 0.5
 0.8
 0.7
 0.7
 0.75
 0.65
 ⋮
 0.75
 0.7
 0.7
 0.7
 0.6
 0.9
 0.8
 0.85
 0.75
 0.7
 0.7
 0.7

In [6]:
function convert_to_mbps(bandwidth)
    try
        if occursin("Mbps", bandwidth)
            return parse(Float64, replace(bandwidth, " Mbps" => ""))
        elseif occursin("Kbps", bandwidth)
            return parse(Float64, replace(bandwidth, " Kbps" => "")) / 1000
        else
            return missing
        end
    catch
        return missing
    end
end

convert_to_mbps (generic function with 1 method)

In [7]:
df.Required_Bandwidth_Mbps = map(x -> convert_to_mbps(string(x)), df.Required_Bandwidth)
df.Allocated_Bandwidth_Mbps = map(x -> convert_to_mbps(string(x)), df.Allocated_Bandwidth)

400-element Vector{Float64}:
 15.0
  0.12
  6.0
  1.5
  3.0
  0.55
  1.0
  0.015
  3.5
  2.0
 14.0
  2.8
  1.2
  ⋮
 15.7
  1.5
  0.4
  0.1
  0.35
  0.0
  6.6
  1.8
 15.8
  1.4
  0.4
  0.1

In [8]:
df.Latency_ms = try
    parse.(Float64, replace.(df.Latency, " ms" => ""))
catch
    fill(missing, nrow(df))
end

400-element Vector{Float64}:
  30.0
  20.0
  40.0
  10.0
  25.0
  50.0
  30.0
 100.0
  35.0
  45.0
  32.0
  24.0
  28.0
   ⋮
  52.0
  57.0
   5.0
   0.0
  78.0
  52.0
  47.0
  61.0
  53.0
  58.0
   5.0
   0.0

In [9]:
df.Signal_Strength_dBm = try
    parse.(Float64, replace.(df.Signal_Strength, " dBm" => ""))
catch
    fill(missing, nrow(df))
end

400-element Vector{Float64}:
  -75.0
  -80.0
  -85.0
  -70.0
  -78.0
  -90.0
  -88.0
  -95.0
  -82.0
  -75.0
  -76.0
  -79.0
  -87.0
    ⋮
  -41.0
 -112.0
  -40.0
 -112.0
 -123.0
  -56.0
  -41.0
 -110.0
  -40.0
 -113.0
  -40.0
 -113.0

In [12]:
CSV.write("Cleaned_Quality_of_Service_5G.csv", df)

"Cleaned_Quality_of_Service_5G.csv"

In [13]:
df = CSV.read("Cleaned_Quality_of_Service_5G.csv", DataFrame)

Row,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation,Required_Bandwidth_Mbps,Allocated_Bandwidth_Mbps,Latency_ms,Signal_Strength_dBm
,String15,String15,String31,String15,String7,String15,String15,Float64,Float64,Float64,Float64,Float64
1,9/3/2023 10:00,User_1,Video_Call,-75 dBm,30 ms,10 Mbps,15 Mbps,0.7,10.0,15.0,30.0,-75.0
2,9/3/2023 10:00,User_2,Voice_Call,-80 dBm,20 ms,100 Kbps,120 Kbps,0.8,0.1,0.12,20.0,-80.0
3,9/3/2023 10:00,User_3,Streaming,-85 dBm,40 ms,5 Mbps,6 Mbps,0.75,5.0,6.0,40.0,-85.0
4,9/3/2023 10:00,User_4,Emergency_Service,-70 dBm,10 ms,1 Mbps,1.5 Mbps,0.9,1.0,1.5,10.0,-70.0
5,9/3/2023 10:00,User_5,Online_Gaming,-78 dBm,25 ms,2 Mbps,3 Mbps,0.85,2.0,3.0,25.0,-78.0
6,9/3/2023 10:00,User_6,Background_Download,-90 dBm,50 ms,500 Kbps,550 Kbps,0.7,0.5,0.55,50.0,-90.0
7,9/3/2023 10:00,User_7,Web_Browsing,-88 dBm,30 ms,1 Mbps,1 Mbps,0.6,1.0,1.0,30.0,-88.0
8,9/3/2023 10:00,User_8,IoT_Temperature,-95 dBm,100 ms,10 Kbps,15 Kbps,0.5,0.01,0.015,100.0,-95.0
9,9/3/2023 10:00,User_9,Video_Streaming,-82 dBm,35 ms,3 Mbps,3.5 Mbps,0.8,3.0,3.5,35.0,-82.0


In [14]:
grouped = combine(groupby(df, :Application_Type), 
                  :Required_Bandwidth_Mbps => mean => :Avg_Required_Bandwidth,
                  :Latency_ms => mean => :Avg_Latency)

Row,Application_Type,Avg_Required_Bandwidth,Avg_Latency
,String31,Float64,Float64
1,Video_Call,11.8147,32.8793
2,Voice_Call,0.1,20.0
3,Streaming,3.35319,41.8511
4,Emergency_Service,0.678723,5.57447
5,Online_Gaming,4.14,29.1111
6,Background_Download,0.547872,56.9787
7,Web_Browsing,0.4875,15.5625
8,IoT_Temperature,0.00384615,102.154
9,Video_Streaming,2.9383,38.8511


In [15]:
grouped.Score = grouped.Avg_Required_Bandwidth .* (1 ./ grouped.Avg_Latency)
total_score = sum(grouped.Score)
grouped.alpha_j = grouped.Score / total_score
sort!(grouped, :alpha_j, rev=true)

Row,Application_Type,Avg_Required_Bandwidth,Avg_Latency,Score,alpha_j
,String31,Float64,Float64,Float64,Float64
1,Video_Call,11.8147,32.8793,0.359334,0.41204
2,Online_Gaming,4.14,29.1111,0.142214,0.163073
3,Emergency_Service,0.678723,5.57447,0.121756,0.139614
4,Streaming,3.35319,41.8511,0.080122,0.091874
5,Video_Streaming,2.9383,38.8511,0.0756298,0.0867229
6,File_Download,2.0,45.0,0.0444444,0.0509634
7,Web_Browsing,0.4875,15.5625,0.0313253,0.03592
8,Background_Download,0.547872,56.9787,0.00961538,0.0110257
9,Voice_Call,0.1,20.0,0.005,0.00573338


In [16]:
println("\nCalculated alpha_j values for each Application Type:")
println(select(grouped, :Application_Type, :Avg_Required_Bandwidth, :Avg_Latency, :alpha_j))


Calculated alpha_j values for each Application Type:
11×4 DataFrame
 Row │ Application_Type     Avg_Required_Bandwidth  Avg_Latency  alpha_j    
     │ String31             Float64                 Float64      Float64    
─────┼──────────────────────────────────────────────────────────────────────
   1 │ Video_Call                      11.8147         32.8793   0.41204
   2 │ Online_Gaming                    4.14           29.1111   0.163073
   3 │ Emergency_Service                0.678723        5.57447  0.139614
   4 │ Streaming                        3.35319        41.8511   0.091874
   5 │ Video_Streaming                  2.9383         38.8511   0.0867229
   6 │ File_Download                    2.0            45.0      0.0509634
   7 │ Web_Browsing                     0.4875         15.5625   0.03592
   8 │ Background_Download              0.547872       56.9787   0.0110257
   9 │ Voice_Call                       0.1            20.0      0.00573338
  10 │ VoIP_Call               

In [17]:
CSV.write("Alpha_Values.csv", select(grouped, :Application_Type, :alpha_j))

"Alpha_Values.csv"

In [22]:
alpha_df = CSV.read("Alpha_Values.csv", DataFrame)
max_latency = maximum(df.Latency_ms)
max_signal_strength = maximum(df.Signal_Strength_dBm)

-40.0

In [23]:
df = leftjoin(df, alpha_df, on=:Application_Type)

Row,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation,Required_Bandwidth_Mbps,Allocated_Bandwidth_Mbps,Latency_ms,Signal_Strength_dBm,alpha_j
,String15,String15,String31,String15,String7,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64?
1,9/3/2023 10:00,User_1,Video_Call,-75 dBm,30 ms,10 Mbps,15 Mbps,0.7,10.0,15.0,30.0,-75.0,0.41204
2,9/3/2023 10:00,User_2,Voice_Call,-80 dBm,20 ms,100 Kbps,120 Kbps,0.8,0.1,0.12,20.0,-80.0,0.00573338
3,9/3/2023 10:00,User_3,Streaming,-85 dBm,40 ms,5 Mbps,6 Mbps,0.75,5.0,6.0,40.0,-85.0,0.091874
4,9/3/2023 10:00,User_4,Emergency_Service,-70 dBm,10 ms,1 Mbps,1.5 Mbps,0.9,1.0,1.5,10.0,-70.0,0.139614
5,9/3/2023 10:00,User_5,Online_Gaming,-78 dBm,25 ms,2 Mbps,3 Mbps,0.85,2.0,3.0,25.0,-78.0,0.163073
6,9/3/2023 10:00,User_6,Background_Download,-90 dBm,50 ms,500 Kbps,550 Kbps,0.7,0.5,0.55,50.0,-90.0,0.0110257
7,9/3/2023 10:00,User_7,Web_Browsing,-88 dBm,30 ms,1 Mbps,1 Mbps,0.6,1.0,1.0,30.0,-88.0,0.03592
8,9/3/2023 10:00,User_8,IoT_Temperature,-95 dBm,100 ms,10 Kbps,15 Kbps,0.5,0.01,0.015,100.0,-95.0,4.31731e-5
9,9/3/2023 10:00,User_9,Video_Streaming,-82 dBm,35 ms,3 Mbps,3.5 Mbps,0.8,3.0,3.5,35.0,-82.0,0.0867229


In [24]:
df.Payoff = df.alpha_j .* 
            df.Allocated_Bandwidth_Mbps .* 
            (1 .- df.Latency_ms / max_latency) .* 
            (1 .+ df.Signal_Strength_dBm / max_signal_strength) .* 
            df.Resource_Allocation

400-element Vector{Float64}:
 9.046146659458696
 0.0013509931272454678
 0.8221679952159857
 0.47119860085722276
 0.9479181077491118
 0.007525062991746312
 0.05015735305079115
 9.934708025795575e-8
 0.504963721189048
 0.12121178906570489
 8.303576011934434
 0.7965158337098218
 0.06631252705783405
 ⋮
 5.180365784133154
 0.16672080893028562
 0.07463024304486114
 0.009554714519961906
 0.0027448061933530604
 0.0
 0.9985945625200799
 0.23481117943368546
 5.060223404663666
 0.15367453311727028
 0.07463024304486114
 0.009617574483909026

In [26]:
CSV.write("Dataset_With_Payoff.csv", df)

"Dataset_With_Payoff.csv"

In [29]:
df = CSV.read("Dataset_With_Payoff.csv", DataFrame)

Row,Timestamp,User_ID,Application_Type,Signal_Strength,Latency,Required_Bandwidth,Allocated_Bandwidth,Resource_Allocation,Required_Bandwidth_Mbps,Allocated_Bandwidth_Mbps,Latency_ms,Signal_Strength_dBm,alpha_j,Payoff
,String15,String15,String31,String15,String7,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,9/3/2023 10:00,User_1,Video_Call,-75 dBm,30 ms,10 Mbps,15 Mbps,0.7,10.0,15.0,30.0,-75.0,0.41204,9.04615
2,9/3/2023 10:00,User_2,Voice_Call,-80 dBm,20 ms,100 Kbps,120 Kbps,0.8,0.1,0.12,20.0,-80.0,0.00573338,0.00135099
3,9/3/2023 10:00,User_3,Streaming,-85 dBm,40 ms,5 Mbps,6 Mbps,0.75,5.0,6.0,40.0,-85.0,0.091874,0.822168
4,9/3/2023 10:00,User_4,Emergency_Service,-70 dBm,10 ms,1 Mbps,1.5 Mbps,0.9,1.0,1.5,10.0,-70.0,0.139614,0.471199
5,9/3/2023 10:00,User_5,Online_Gaming,-78 dBm,25 ms,2 Mbps,3 Mbps,0.85,2.0,3.0,25.0,-78.0,0.163073,0.947918
6,9/3/2023 10:00,User_6,Background_Download,-90 dBm,50 ms,500 Kbps,550 Kbps,0.7,0.5,0.55,50.0,-90.0,0.0110257,0.00752506
7,9/3/2023 10:00,User_7,Web_Browsing,-88 dBm,30 ms,1 Mbps,1 Mbps,0.6,1.0,1.0,30.0,-88.0,0.03592,0.0501574
8,9/3/2023 10:00,User_8,IoT_Temperature,-95 dBm,100 ms,10 Kbps,15 Kbps,0.5,0.01,0.015,100.0,-95.0,4.31731e-5,9.93471e-8
9,9/3/2023 10:00,User_9,Video_Streaming,-82 dBm,35 ms,3 Mbps,3.5 Mbps,0.8,3.0,3.5,35.0,-82.0,0.0867229,0.504964


In [30]:
alpha_df = CSV.read("Alpha_Values.csv", DataFrame)

Row,Application_Type,alpha_j
,String31,Float64
1,Video_Call,0.41204
2,Online_Gaming,0.163073
3,Emergency_Service,0.139614
4,Streaming,0.091874
5,Video_Streaming,0.0867229
6,File_Download,0.0509634
7,Web_Browsing,0.03592
8,Background_Download,0.0110257
9,Voice_Call,0.00573338


In [31]:
strategies = [
    (bandwidth=10.0, latency=10.0),  # High Demand
    (bandwidth=0.1, latency=50.0)    # Low Demand
]

2-element Vector{@NamedTuple{bandwidth::Float64, latency::Float64}}:
 (bandwidth = 10.0, latency = 10.0)
 (bandwidth = 0.1, latency = 50.0)

In [32]:
function compute_payoff(alpha_j, allocated_bandwidth, latency, signal_strength, resource_allocation, max_latency, max_signal_strength)
    return alpha_j * allocated_bandwidth * (1 - latency / max_latency) * (1 + signal_strength / max_signal_strength) * resource_allocation
end

compute_payoff (generic function with 1 method)

In [33]:
total_bandwidth = 1000.0  # Total available bandwidth (Mbps)
function get_allocated_bandwidth(requested_bandwidths)
    total_requested = sum(requested_bandwidths)
    if total_requested == 0
        return zeros(length(requested_bandwidths))
    end
    return [min(bw, total_bandwidth * bw / total_requested) for bw in requested_bandwidths]
end

get_allocated_bandwidth (generic function with 1 method)

In [51]:
function compute_expected_payoff(player_idx, strategy_idx, p, df, max_latency, max_signal_strength, n_samples=500)
    n_players = nrow(df)
    payoff_sum = 0.0
    strategy = strategies[strategy_idx]
    
    for _ in 1:n_samples
        # Simulate a strategy profile
        requested_bandwidths = zeros(n_players)
        for i in 1:n_players
            if i == player_idx
                requested_bandwidths[i] = strategy.bandwidth
            else
                # Sample strategy based on p[i]
                if rand() < p[i]
                    requested_bandwidths[i] = strategies[1].bandwidth  # High Demand
                else
                    requested_bandwidths[i] = strategies[2].bandwidth  # Low Demand
                end
            end
        end
        allocated = get_allocated_bandwidth(requested_bandwidths)
        payoff = compute_payoff(df.alpha_j[player_idx],allocated[player_idx],strategy.latency,df.Signal_Strength_dBm[player_idx],df.Resource_Allocation[player_idx],max_latency,max_signal_strength)
        payoff_sum += payoff
    end
    
    return payoff_sum / n_samples
end

compute_expected_payoff (generic function with 2 methods)

In [57]:
function best_response_dynamics(df, max_iterations=1000, eta=0.5, epsilon=1e-5)
    n_players = nrow(df)
    max_latency = maximum(df.Latency_ms)
    max_signal_strength = maximum(df.Signal_Strength_dBm)
    p = rand(n_players) 
    for iteration in 1:max_iterations
        p_old = copy(p)
        max_change = 0.0
        for i in 1:n_players
            payoff_high = compute_expected_payoff(i, 1, p, df, max_latency, max_signal_strength)
            payoff_low = compute_expected_payoff(i, 2, p, df, max_latency, max_signal_strength)
            best_response = payoff_high > payoff_low ? 1.0 : 0.0
            p[i] = (1 - eta) * p[i] + eta * best_response
            max_change = max(max_change, abs(p[i] - p_old[i]))
        end
        if max_change < epsilon
            println("Converged after $iteration iterations.")
            return p
        end
        if iteration % 100 == 0
            println("Iteration $iteration, max change: $max_change")
        end
    end
    println("Reached maximum iterations ($max_iterations) without convergence.")
    return p
end

best_response_dynamics (generic function with 4 methods)

In [40]:
using Random

In [58]:
Random.seed!(123)  # For reproducibility
mixed_strategies = best_response_dynamics(df)

Converged after 17 iterations.


400-element Vector{Float64}:
 0.9999992851229766
 0.9999957541941258
 0.9999980596418849
 0.999996277489014
 0.9999943073203197
 0.9999949199793731
 0.9999956308584661
 0.999998989465334
 0.9999931269349277
 0.9999933264725318
 0.9999976517384495
 0.9999934124105464
 0.9999926154836529
 ⋮
 0.9999976374895205
 0.9999941641750156
 0.999999751941683
 0.9999949242228812
 0.9999977364658437
 0.999999124239386
 0.9999980366893346
 0.9999990993914555
 0.9999976426722104
 0.9999951957017599
 0.9999961520530904
 0.9999966019652285

In [56]:
println("\nMixed Strategy Nash Equilibrium (probability of High Demand):")
for i in 1:nrow(df)   
    println("User_$(df.User_ID[i]): p = $(mixed_strategies[i])")
end


Mixed Strategy Nash Equilibrium (probability of High Demand):
User_User_1: p = 0.999919464122044
User_User_2: p = 0.9995216803834145
User_User_3: p = 0.9997814051378818
User_User_4: p = 0.9995806331988915
User_User_5: p = 0.9993586799673079
User_User_6: p = 0.9994277002786879
User_User_7: p = 0.9995077857619458
User_User_8: p = 0.9998861562284477
User_User_9: p = 0.9992257013279521
User_User_10: p = 0.9992481806294389
User_User_11: p = 0.9997354519736048
User_User_12: p = 0.9992578621455982
User_User_13: p = 0.9991680827780431
User_User_14: p = 0.9994417912397596
User_User_15: p = 0.9999401203670811
User_User_16: p = 0.999965132997938
User_User_17: p = 0.9998058308040463
User_User_18: p = 0.9992982616515859
User_User_19: p = 0.999395787993956
User_User_20: p = 0.9992695543898458
User_User_21: p = 0.9999084945312513
User_User_22: p = 0.9994455058763659
User_User_23: p = 0.9992537767628545
User_User_24: p = 0.9999494036282237
User_User_25: p = 0.9991895734442094
User_User_26: p = 0.9993

User_User_272: p = 0.9992181615836319
User_User_273: p = 0.9994626034300603
User_User_274: p = 0.9996987939329499
User_User_275: p = 0.9996375057978487
User_User_276: p = 0.9992081116900494
User_User_277: p = 0.9999914729479545
User_User_278: p = 0.9991951830544348
User_User_279: p = 0.9997742863575738
User_User_280: p = 0.999899782938453
User_User_281: p = 0.9994401453998638
User_User_282: p = 0.9997521233993303
User_User_283: p = 0.9991793631389261
User_User_284: p = 0.9997718189266224
User_User_285: p = 0.999644799256787
User_User_286: p = 0.9996285084502549
User_User_287: p = 0.9996669680676651
User_User_288: p = 0.9998062088451063
User_User_289: p = 0.999188329689144
User_User_290: p = 0.9996917943559492
User_User_291: p = 0.9994712109978009
User_User_292: p = 0.9994203362162443
User_User_293: p = 0.9995764344675704
User_User_294: p = 0.9995908874637611
User_User_295: p = 0.9991608677141445
User_User_296: p = 0.9995541647056561
User_User_297: p = 0.9993770738727948
User_User_298: 

In [59]:
results_df = DataFrame(User_ID=df.User_ID, Prob_High_Demand=mixed_strategies)
CSV.write("Mixed_Strategy_Equilibrium.csv", results_df)
println("\nResults saved to Mixed_Strategy_Equilibrium.csv")


Results saved to Mixed_Strategy_Equilibrium.csv
